In [0]:
dbutils.fs.ls("dbfs:/databricks-datasets/definitive-guide/data/retail-data/by-day/")

In [0]:
# --SHOW VOLUMES in dev.etl
# -- CREATE VOLUME IF NOT EXISTS dev.etl.landing
# -- COMMENT 'Volume for source file'
dbutils.fs.mkdirs("/Volumes/dev/etl/landing/files")
dbutils.fs.mkdirs("/Volumes/dev/etl/landing/autoloader/schemas")

In [0]:
%sql
SELECT * FROM dev.etl.orders_agg_gold

In [0]:
%sql
SELECT * FROM dev.etl.orders_autoloader_bronze

In [0]:
%sql
-- create volume if not exists dev.bronze.landing
SELECT b.c_mktsegment, a.o_custkey, b.c_custkey 
FROM dev.etl.orders_union_bronze a
INNER JOIN dev.etl.customer_bronze b ON a.o_custkey = b.c_custkey  
WHERE a.o_orderkey IN (5611649, 5611650)

In [0]:
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/checkpoint/autoloader")

In [0]:
dbutils.fs.cp("dbfs:/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/01/2010-12-01.csv")
dbutils.fs.cp("dbfs:/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/02/2010-12-02.csv")
dbutils.fs.cp("dbfs:/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/03/2010-12-03.csv")

In [0]:
dbutils.fs.cp("dbfs:/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-05.csv", "/Volumes/dev/bronze/landing/autoloader_input/2010/12/05/2010-12-05.csv")

In [0]:
df=(
    spark
    .readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("pathGlobFilter", "*.csv")
    .option("header", "true")
    .option("cloudFiles.schemaHints", "Quantity int, Unitprice double")
    .option("cloudFiles.schemaLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1/")
    .load("/Volumes/dev/bronze/landing/autoloader_input/*/")
)
df.printSchema()

In [0]:
from pyspark.sql.functions import col
(
    df
    .withColumn("_file", col("_metadata.file_name"))
    .writeStream
    .format("delta")
    .option("checkpointLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1/")
    .outputMode("append")
    .trigger(availableNow=True)
    .table("dev.bronze.invoice_1")
)

In [0]:
%sql
SELECT
 _file, count(1)
FROM
  dev.bronze.invoice_1
 GROUP BY _file 